# Cheatsheet

## How to Set Up Credentials for AWS 

In [ ]:
# step 1: ensure you have AWS CLI installed and configured with SSO
# step 2: login to your AWS SSO profile
aws sso login --profile myisb01_IsbUsersPS-371061166839

# step 3: verify you can access AWS with the profile
aws sts get-caller-identity --profile myisb01_IsbUsersPS-371061166839

# step 4: set the AWS_PROFILE environment variable to use the profile
export AWS_PROFILE=myisb01_IsbUsersPS-371061166839

## To Deploy Frontend and Backend

In [ ]:
# backend deployment in terminal
source venv/bin/activate
# only if your venv is missing dependencies
## pip install -r requirements.txt
uvicorn backend.main:app --host 0.0.0.0 --port 8000

# frontend deployment in a new terminal
cd frontend
# only if your node modules are missing
## npm install 
npm start


## To Set Up Lambda Functions (Attempting)

In [ ]:
make ecr-create REGION=us-east-1
make login-ecr REGION=us-east-1
make push-budget REGION=us-east-1
make push-cost REGION=us-east-1
make lambda-create-budget REGION=us-east-1
make lambda-create-cost REGION=us-east-1

In [ ]:
# create or update the lambda
# first time
make lambda-create-cost REGION=us-east-1

# subsequent updates
make lambda-update-cost REGION=us-east-1

In [ ]:
# updating after code changes

# rebuild/push:
make push-budget REGION=us-east-1 (or push-budget-nocache)
make push-cost REGION=us-east-1 (or push-cost-nocache)

# update lambda to new image:
make lambda-update-budget REGION=us-east-1
make lambda-update-cost REGION=us-east-1

In [ ]:
# push images with buildx (gzip) and architecture set to match your lambda
make push-budget-buildx REGION=us-east-1 ARCH=arm64
make push-cost-buildx REGION=us-east-1 ARCH=arm64

# create functions
make lambda-create-budget REGION=us-east-1 ARCH=arm64
make lambda-create-cost REGION=us-east-1 ARCH=arm64

# expose URLs
make url-budget REGION=us-east-1 && make print-url-budget REGION=us-east-1
make url-cost REGION=us-east-1 && make print-url-cost REGION=us-east-1

# if still hit the media-type error, re-run with no-cache to avoid reusing problematic local layers
docker buildx build --no-cache --platform linux/arm64 -f deploy/Dockerfile.budget -t $ECR_BUDGET:latest --provenance=false --sbom=false --output type=image,compression=gzip --push .
docker buildx build --no-cache --platform linux/arm64 -f deploy/Dockerfile.cost -t $ECR_COST:latest --provenance=false --sbom=false --output type=image,compression=gzip --push .

# after deploy
# tail logs to confirm execution
aws logs tail /aws/lambda/bto-budget-estimator --follow --region us-east-1
aws logs tail /aws/lambda/bto-cost-estimator --follow --region us-east-1